In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow.keras.layers as tkl
import cv2
from skimage.util import random_noise
from sklearn.model_selection import train_test_split
from itertools import chain
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import glob
import os


In [10]:
! cd . ; pwd; ls; 
!cd lfw_funneled; pwd; ls

In [11]:
!tar -xvf '../input/lfwpeople/lfw-funneled.tgz' -C './'

In [12]:
source_path = './lfw_funneled'
# destination_path = './Pdataset'
filenames = []

# for file in source_path:
# #    filenames.append(glob.glob('source_path/file/*jpg'))
#     print(file)
# len(filenames)
file_path = os.listdir(source_path)

for file in file_path:
    
    filenames.append(glob.glob(os.path.join(source_path, f'{file}/*.jpg')))

filenames = list(chain.from_iterable(filenames))
random.shuffle(filenames)
filenames_backup = filenames

### Showing an image before and after adding some noises:

In [13]:
img_path = filenames[1000]
print(img_path)
img = cv2.imread(img_path)
# img = img.astype(np.float64) / 255
# img = Image.convert(img, )
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
noisy = random_noise(img, mode = 's&p', amount=0.2)
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[0].set_title('Original')
ax[1].imshow(noisy)
ax[1].set_title('Noisy')
# plt.imshow(noisy)

### Length of the dataset:

In [14]:
print(np.array(filenames).shape)

### Spliting the train and test set:

In [15]:
test_filenames = filenames[-4001:-1]
del filenames[-4001:-1]
train_filenames = filenames
print(f'Size of training set is: {len(train_filenames)}', end='\n----------------------------\n')
print(f'Size of test set is: {len(test_filenames)}', end='\n----------------------------\n')

In [16]:
train_img, val_img, _, _ = train_test_split(train_filenames, train_filenames, test_size = 0.2, shuffle = True, random_state = 0)

In [17]:
print(f'Size of training set is: {len(train_img)}', end='\n----------------------------\n')
print(f'Size of validation set is: {len(val_img)}', end='\n----------------------------\n')
print(f'Total: {len(train_img) + len(val_img)}')

In [18]:
WIDTH = 256
HEIGHT = 256
n_channels = 3
BATCH_SIZE = 128

In [33]:
class  DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, orig_filenames, batch_size = BATCH_SIZE, shuffle = True):
        self.orig_filenames = orig_filenames
        self.noisy_filenames = orig_filenames
        self.filenames = list(zip(self.orig_filenames, self.noisy_filenames))
        self.batch_size = BATCH_SIZE
        self.shuffle = shuffle
    
    def __len__(self):
        return (len(self.orig_filenames) // self.batch_size)
    
    def __getitem__(self, idx):
        batch = self.filenames[idx * self.batch_size : (idx + 1) * self.batch_size]
        X, Y = self.__data_generation(batch)

        return X, Y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.orig_filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, batch):
        orig = []
        noisy = []
        
        for orig_file, _ in batch:
          # image = self.colored_image_loader(orig_file)
          image = cv2.imread(orig_file) / 255
#           image = tf.image.convert_image_dtype(image, tf.float32) / 255
          image = cv2.resize(image, (WIDTH, HEIGHT))
#           image = tf.image.convert_image_dtype(image, tf.float32)
            
          # noisy = cv2.imread(mask_file)
          # mask = cv2.resize(mask, (WIDTH, HEIGHT))[:, :, 2]

#           orig.append(cv2.resize(image, (252, 252)))
          orig.append(image)
          
          noisy.append(self.make_noisy(image))
          
        return  np.array(noisy), np.array(orig)

    # def colored_image_loader(self, image_filename):
    #   img = cv2.imread(image_filename)
    #   img = cv2.resize(img, (WIDTH, HEIGHT))
    #   img = tf.image.convert_image_dtype(img, tf.float32)
      
      # return img

    def make_noisy(self, image):
      # image = cv2.imread(img_file)
      image_ = image
      noisy_image = random_noise(image_, mode = 's&p', amount = 0.2)
      
      return noisy_image

In [34]:
# class  DataGenerator(tf.keras.utils.Sequence):
    
#     def __init__(self, orig_filenames, batch_size = BATCH_SIZE, shuffle = True):

train_generator = DataGenerator(train_img, BATCH_SIZE)
val_generator = DataGenerator(val_img, BATCH_SIZE)

In [35]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Conv2D, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [37]:
import tensorflow.keras.layers as tkl
def conv_block(inputs = None, n_filters = 32, kernel_size = 3, batch_norm = True):
    
    '''First layer'''
    
    conv = tkl.Conv2D(filters = n_filters,
                      kernel_size = kernel_size,
                      padding = 'same',
                      kernel_initializer = 'he_normal')(inputs)
    
    if batch_norm:
        conv = tkl.BatchNormalization()(conv)
    
    conv = tkl.Activation('relu')(conv)
    
    '''Second layer'''
    
    conv = tkl.Conv2D(filters = n_filters,
                     kernel_size = kernel_size,
                     padding = 'same',
                     kernel_initializer = 'he_normal')(conv)
    
    if batch_norm:
        conv = tkl.BatchNormalization()(conv)
    
    conv = tkl.Activation('relu')(conv)
        
    return conv


def conv_transpose_block(n_filters, kernel_size = 3):
    conv_transpose = tkl.Conv2DTranspose(n_filters, kernel_size = kernel_size, strides = (2,2), padding = 'same')
    
    return conv_transpose
    

def u_net(n_filters = 32, dropout_prob = 0.1, batch_norm = True):
    input_size = (WIDTH, HEIGHT, n_channels)
    
    input_img = tf.keras.Input(input_size, name = 'image' )
    
    c1 = conv_block(input_img, n_filters, kernel_size = 3)
    p1 = tkl.MaxPooling2D((2,2))(c1)
    p1 = tkl.Dropout(dropout_prob)(p1)
    
    c2 = conv_block(p1, n_filters * 2, kernel_size = 3)
    p2 = tkl.MaxPooling2D((2,2))(c2)
    p2 = tkl.Dropout(dropout_prob)(p2)
    
    c3 = conv_block(p2, n_filters * 4, kernel_size = 3)
    p3 = tkl.MaxPooling2D((2,2))(c3)
    p3 = tkl.Dropout(dropout_prob)(p3)
    
    c4 = conv_block(p3, n_filters * 8, kernel_size = 3)
    p4 = tkl.MaxPooling2D((2,2))(c4)
    p4 = tkl.Dropout(dropout_prob)(p4)
    
    c5 = conv_block(p4, n_filters * 16 , kernel_size = 3)
    
    ''' UpSampling '''
    
    u6 = conv_transpose_block(n_filters * 8, kernel_size = 3)(c5)  #----> 256
    u6 = tkl.concatenate([u6, c4])     #----> 512
    u6 = tkl.Dropout(dropout_prob)(u6)
    c6 = conv_block(u6, n_filters * 8, kernel_size = 3)   #----> 256
    
    u7 = conv_transpose_block(n_filters * 4)(c6)   #----> 128
    u7 = tkl.concatenate([u7, c3])   #----> 256
    u7 = tkl.Dropout(dropout_prob)(u7)
    c7 = conv_block(u7, n_filters * 4, kernel_size = 3)   #----> 128
    
    u8 = conv_transpose_block(n_filters * 2, kernel_size = 3)(c7)   #----> 64
    u8 = tkl.concatenate([u8, c2])   #----> 128
    u8 = tkl.Dropout(dropout_prob)(u8)
    c8 = conv_block(u8, n_filters * 2, kernel_size = 3)   #----> 64
    
    u9 = conv_transpose_block(n_filters, kernel_size = 3)(c8)   #----> 32
    u9 = tkl.concatenate([u9, c1])   #----> 64
    u9 = tkl.Dropout(dropout_prob)(u9)
    c9 = conv_block(u9, n_filters , kernel_size = 3)   #----> 32
    
    output = tkl.Conv2D(3, (1,1), activation = 'sigmoid')(c9)
    
    model = tf.keras.Model(inputs = [input_img], outputs = [output])

    return model

In [38]:
# model = create_model()
model = u_net()
model.summary()

In [39]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [40]:
# gaussian_early_stop = EarlyStopping(monitor='loss', patience=3)
# x, y = train_generator
history = model.fit(train_generator,
                              validation_data = val_generator,
                              use_multiprocessing = True,
                              workers = 6, epochs=50)
# , callbacks=[gaussian_early_stop]

# history = model.fit(train_generator, epochs=50)

In [ ]:
model.save('UnetDenoising.h5')

In [32]:
path = '/kaggle/input/stanford-cars-dataset/cars_test/cars_test/'
test_data = list(glob.glob(path + '*.jpg'))
# print(len(test_data))

evaluation_set = test_data[:64]
test_img_path = test_data[104]
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img, (WIDTH, HEIGHT)) / 255
test_img = random_noise(test_img, mode = 'gaussian')
plt.imshow(test_img)
print(test_img.shape)

In [ ]:
model.predict(test_img)

In [ ]:
eval_set = []
for i in range(64):
    img = cv2.imread(evaluation_set[i])
    img = cv2.resize(img, (WIDTH, HEIGHT)) / 255
    img = random_noise(img, mode = 'gaussian')
    eval_set.append(img)
eval_set = np.array(eval_set)

# test_generator = DataGenerator(evaluation_set)
# x, y = test_generator[2]
predicted = model.predict(eval_set)

In [ ]:
predicted.shape

In [ ]:
# plt.imshow(predicted[10])
predicted_ = cv2.resize(predicted[11], (500, 500))
# fig, ax = plt.subplots(1,3)
prepared_img = cv2.imread(evaluation_set[11]) / 255
prepared_img = cv2.resize(prepared_img, (500, 500))
noisy_orig = random_noise(prepared_img, mode = 'gaussian')
# ax[0].imshow(prepared_img)
# ax[1].imshow(predicted_)
# ax[2].imshow(noisy_orig)
